In [20]:
from mp_api.client import MPRester
from tqdm.notebook import tqdm
import pandas as pd
from matminer.featurizers.structure.composite import JarvisCFID

API_KEY = 'm79FCfWQZzWuPVJsCRMwXp87Y0ke7jGZ'
with MPRester(API_KEY) as mpr:
    print(mpr.materials.summary.available_fields)

['builder_meta', 'nsites', 'elements', 'nelements', 'composition', 'composition_reduced', 'formula_pretty', 'formula_anonymous', 'chemsys', 'volume', 'density', 'density_atomic', 'symmetry', 'property_name', 'material_id', 'deprecated', 'deprecation_reasons', 'last_updated', 'origins', 'warnings', 'structure', 'task_ids', 'uncorrected_energy_per_atom', 'energy_per_atom', 'formation_energy_per_atom', 'energy_above_hull', 'is_stable', 'equilibrium_reaction_energy_per_atom', 'decomposes_to', 'xas', 'grain_boundaries', 'band_gap', 'cbm', 'vbm', 'efermi', 'is_gap_direct', 'is_metal', 'es_source_calc_id', 'bandstructure', 'dos', 'dos_energy_up', 'dos_energy_down', 'is_magnetic', 'ordering', 'total_magnetization', 'total_magnetization_normalized_vol', 'total_magnetization_normalized_formula_units', 'num_magnetic_sites', 'num_unique_magnetic_sites', 'types_of_magnetic_species', 'k_voigt', 'k_reuss', 'k_vrh', 'g_voigt', 'g_reuss', 'g_vrh', 'universal_anisotropy', 'homogeneous_poisson', 'e_total

Retrieving SummaryDoc documents:   0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

C:\Users\joeya\anaconda3\Lib\site-packages\mp_api\client\mprester.py:227: UserWarning: mpcontribs-client not installed. Install the package to query MPContribs data, or construct pourbaix diagrams: 'pip install mpcontribs-client'
  warnings.warn(


Retrieving SummaryDoc documents:   0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

,jml_bp_mult_atom_rad,jml_hfus_add_bp,jml_elec_aff_mult_voro_coord,jml_mol_vol_subs_atom_mass,jml_is_halogen,jml_atom_mass_subs_first_ion_en,jml_row,jml_mol_vol_mult_atom_mass,jml_voro_coord_divi_therm_cond,jml_voro_coord_subs_mp,...,total_magnetization_normalized_vol,total_magnetization,volume,nelements,nsites,crystal_system,number,symbol,point_group,structure
0,4121.6,2944.2100,6.43,-44.7661,0.0,45.22959,4.0,375.931803,0.106383,-2170.0,...,0.000000e+00,0.000000e+00,23.002393,1,2,Orthorhombic,63,Cmcm,mmm,"[[-1.82823127 -1.14705241 -3.04823305] Cr, [-6..."
1,4121.6,2944.2100,6.43,-44.7661,0.0,45.22959,4.0,375.931803,0.106383,-2170.0,...,2.292793e-08,6.000000e-07,26.168958,1,2,Cubic,229,Im-3m,m-3m,[[-1.66945527e-05 2.90100006e-05 2.57116964e...
2,4387.6,3134.1381,1.57,-48.7550,0.0,47.94260,4.0,395.941050,0.125000,-1801.0,...,1.166675e-06,2.300000e-05,19.714143,1,2,Hexagonal,194,P6_3/mmc,6/mmm,[[-1.54707902e-06 1.40536193e+00 9.60483617e...
3,4387.6,3134.1381,1.57,-48.7550,0.0,47.94260,4.0,395.941050,0.125000,-1801.0,...,2.254071e-01,5.289892e+00,23.468168,1,2,Cubic,229,Im-3m,mmm,"[[1.16907043 2.0252321 0.8292113 ] Fe, [ 1.16..."


In [ ]:
jarvis = JarvisCFID()

# This is a helper function to shorten lists during
# the live presentation of this lesson for better readibility.
# You can ignore it.


def shortlist(long_list, n=5):
    print("First {} of {} items:".format(min(n, 5), len(long_list)))
    for item in long_list[0:n]:
        print(item)


JarvisFeatures = []
Formulas = []
material_id = []
nelements = []
total_mag_norm_fu = []
total_mag_norm_vol = []
total_mag = []
nsites = []
volume = []
stru = []
crystal_system = []
number = []
symbol = []
point_group = []

with MPRester(API_KEY) as mpr:
    docTest1 = mpr.materials.summary.search(material_ids=["mp-90", 'mp-1059289'], fields=["formula_pretty", "elements", "structure", "volume", "nelements",
                                            "symmetry", "total_magnetization", "total_magnetization_normalized_vol", "total_magnetization_normalized_formula_units", "nsites", "material_id"])
for entry in tqdm(docTest1):
    Formulas.append(entry.formula_pretty)
    material_id.append(entry.material_id)
    total_mag_norm_fu.append(
        entry.total_magnetization_normalized_formula_units)
    total_mag_norm_vol.append(entry.total_magnetization_normalized_vol)
    total_mag.append(entry.total_magnetization)
    crystal_system.append(entry.symmetry.crystal_system)
    volume.append(entry.volume)
    nelements.append(entry.nelements)
    number.append(entry.symmetry.number)
    symbol.append(entry.symmetry.symbol)
    point_group.append(entry.symmetry.point_group)
    nsites.append(entry.nsites)
    stru.append(entry.structure)
    JarvisFeatures.append(jarvis.featurize(entry.structure))

df = pd.DataFrame(JarvisFeatures)
df.columns = jarvis.feature_labels()
df['material_id'] = material_id
df['formula'] = Formulas
df['total_magnetization_normalized_formula_units'] = total_mag_norm_fu
df['total_magnetization_normalized_vol'] = total_mag_norm_vol
df['total_magnetization'] = total_mag
df['volume'] = volume
df['nelements'] = nelements
df['nsites'] = nsites
df['crystal_system'] = crystal_system
df['number'] = number
df['symbol'] = symbol
df['point_group'] = point_group
df['structure'] = stru

with MPRester(API_KEY) as mpr:
    docTest2 = mpr.materials.summary.search(material_ids=['mp-13', 'mp-136'], fields=["formula_pretty", "elements", "structure", "volume", "nelements",
                                            "symmetry", "total_magnetization", "total_magnetization_normalized_vol", "total_magnetization_normalized_formula_units", "nsites", "material_id"])
for entry in tqdm(docTest2):
    Formulas.append(entry.formula_pretty)
    material_id.append(entry.material_id)
    total_mag_norm_fu.append(
        entry.total_magnetization_normalized_formula_units)
    total_mag_norm_vol.append(entry.total_magnetization_normalized_vol)
    total_mag.append(entry.total_magnetization)
    crystal_system.append(entry.symmetry.crystal_system)
    volume.append(entry.volume)
    nelements.append(entry.nelements)
    number.append(entry.symmetry.number)
    symbol.append(entry.symmetry.symbol)
    point_group.append(entry.symmetry.point_group)
    nsites.append(entry.nsites)
    stru.append(entry.structure)
    JarvisFeatures.append(jarvis.featurize(entry.structure))

df = pd.DataFrame(JarvisFeatures)
df.columns = jarvis.feature_labels()
df['material_id'] = material_id
df['formula'] = Formulas
df['total_magnetization_normalized_formula_units'] = total_mag_norm_fu
df['total_magnetization_normalized_vol'] = total_mag_norm_vol
df['total_magnetization'] = total_mag
df['volume'] = volume
df['nelements'] = nelements
df['nsites'] = nsites
df['crystal_system'] = crystal_system
df['number'] = number
df['symbol'] = symbol
df['point_group'] = point_group
df['structure'] = stru


display(df)
df.to_csv(r'C:\Users\joeya\Documents\Fokwa Group\ML_tutorial\intermetallics\JarvisTEST.csv', index=False)

In [14]:
docCr = mpr.materials.summary.search(elements=["Cr"], exclude_elements=["O",'Eu','Tc','Ac',"Pu","Pm","U","Np","Pa","Pu","Th",'H','S','F','Cl','Br','I','At','Xe'] 
                                 ,fields=["structure"]) 
docMn = mpr.materials.summary.search(elements=["Mn"], exclude_elements=["O",'Eu','Tc','Ac',"Pu","Pm","U","Np","Pa","Pu","Th",'H','S','F','Cl','Br','I','At','Xe'] 
                                 ,fields=["structure"]) 
docFe = mpr.materials.summary.search(elements=["Fe"], exclude_elements=["O",'Eu','Tc','Ac',"Pu","Pm","U","Np","Pa","Pu","Th",'H','S','F','Cl','Br','I','At','Xe'] 
                                 ,fields=["structure"]) 
docCo = mpr.materials.summary.search(elements=["Co"], exclude_elements=["O",'Eu','Tc','Ac',"Pu","Pm","U","Np","Pa","Pu","Th",'H','S','F','Cl','Br','I','At','Xe'] 
                                 ,fields=["structure"]) 
docNi = mpr.materials.summary.search(elements=["Ni"], exclude_elements=["O",'Eu','Tc','Ac',"Pu","Pm","U","Np","Pa","Pu","Th",'H','S','F','Cl','Br','I','At','Xe'] 
                                 ,fields=["structure"]) 

doctest  = docCr + docMn + docFe + docCo + docNi
print(len(doctest))


Retrieving SummaryDoc documents:   0%|          | 0/1434 [00:00<?, ?it/s]

Retrieving SummaryDoc documents:   0%|          | 0/2313 [00:00<?, ?it/s]

Retrieving SummaryDoc documents:   0%|          | 0/2795 [00:00<?, ?it/s]

Retrieving SummaryDoc documents:   0%|          | 0/2893 [00:00<?, ?it/s]

Retrieving SummaryDoc documents:   0%|          | 0/3592 [00:00<?, ?it/s]

13027


In [22]:
#Test worked, so now doing it on the full dataset
from mp_api.client import MPRester
from tqdm.notebook import tqdm
API_KEY = 'm79FCfWQZzWuPVJsCRMwXp87Y0ke7jGZ'
with MPRester(API_KEY) as mpr:
    print(mpr.materials.summary.available_fields)

import pandas as pd
from matminer.featurizers.structure.composite import JarvisCFID
jarvis = JarvisCFID()    

#This is a helper function to shorten lists during
#the live presentation of this lesson for better readibility.
#You can ignore it.
def shortlist(long_list, n=5):
    print("First {} of {} items:".format(min(n, 5), len(long_list)))
    for item in long_list[0:n]:
        print(item)
        
docCr = mpr.materials.summary.search(elements=["Cr"], exclude_elements=["O",'Eu','Tc','Ac',"Pu","Pm","U","Np","Pa","Pu","Th",'H','S','F','Cl','Br','I','At','Xe'],
                                 fields=["formula_pretty","elements","structure", "volume",'elements'
                                          , "nelements","symmetry"
                                        ,"total_magnetization","total_magnetization_normalized_vol"
                                        ,"total_magnetization_normalized_formula_units"
                                   ,"nsites","material_id"]) 
for entry in tqdm(docCr):
    Formulas.append(entry.formula_pretty)
    material_id.append(entry.material_id)
    total_mag_norm_fu.append(entry.total_magnetization_normalized_formula_units)
    total_mag_norm_vol.append(entry.total_magnetization_normalized_vol)
    total_mag.append(entry.total_magnetization)
    crystal_system.append(entry.symmetry.crystal_system)
    volume.append(entry.volume)
    nelements.append(entry.nelements)
    number.append(entry.symmetry.number)
    symbol.append(entry.symmetry.symbol)
    point_group.append(entry.symmetry.point_group)
    nsites.append(entry.nsites)
    stru.append(entry.structure)
    JarvisFeatures.append(jarvis.featurize(entry.structure))
    
df=pd.DataFrame(JarvisFeatures)
df.columns = jarvis.feature_labels()
df['material_id'] = material_id
df['formula'] = Formulas
df['total_magnetization_normalized_formula_units'] = total_mag_norm_fu
df['total_magnetization_normalized_vol'] = total_mag_norm_vol
df['total_magnetization'] = total_mag
df['volume'] = volume
df['nelements'] = nelements
df['nsites'] = nsites
df['crystal_system'] = crystal_system
df['number'] = number
df['symbol'] = symbol
df['point_group'] = point_group
df['structure'] = stru

df.to_csv(r'C:\Users\joeya\Documents\Fokwa Group\ML_tutorial\intermetallics\intermetallics_dataset_JarvisCr.csv', index=False)

docMn = mpr.materials.summary.search(elements=["Mn"], exclude_elements=["O",'Eu','Tc','Ac',"Pu","Pm","U","Np","Pa","Pu","Th",'H','S','F','Cl','Br','I','At','Xe'] 
                                 ,fields=["formula_pretty","elements","structure", "volume", 'elements'
                                          , "nelements","symmetry"
                                        ,"total_magnetization","total_magnetization_normalized_vol"
                                        ,"total_magnetization_normalized_formula_units"
                                   ,"nsites","material_id"]) 
for entry in tqdm(docMn):
    Formulas.append(entry.formula_pretty)
    material_id.append(entry.material_id)
    total_mag_norm_fu.append(entry.total_magnetization_normalized_formula_units)
    total_mag_norm_vol.append(entry.total_magnetization_normalized_vol)
    total_mag.append(entry.total_magnetization)
    crystal_system.append(entry.symmetry.crystal_system)
    volume.append(entry.volume)
    nelements.append(entry.nelements)
    number.append(entry.symmetry.number)
    symbol.append(entry.symmetry.symbol)
    point_group.append(entry.symmetry.point_group)
    nsites.append(entry.nsites)
    stru.append(entry.structure)
    JarvisFeatures.append(jarvis.featurize(entry.structure))
    
df=pd.DataFrame(JarvisFeatures)
df.columns = jarvis.feature_labels()
df['material_id'] = material_id
df['formula'] = Formulas
df['total_magnetization_normalized_formula_units'] = total_mag_norm_fu
df['total_magnetization_normalized_vol'] = total_mag_norm_vol
df['total_magnetization'] = total_mag
df['volume'] = volume
df['nelements'] = nelements
df['nsites'] = nsites
df['crystal_system'] = crystal_system
df['number'] = number
df['symbol'] = symbol
df['point_group'] = point_group
df['structure'] = stru

df.to_csv(r'C:\Users\joeya\Documents\Fokwa Group\ML_tutorial\intermetallics\intermetallics_dataset_JarvisCrMn.csv', index=False)

docFe = mpr.materials.summary.search(elements=["Fe"], exclude_elements=["O",'Eu','Tc','Ac',"Pu","Pm","U","Np","Pa","Pu","Th",'H','S','F','Cl','Br','I','At','Xe'] 
                                 ,fields=["formula_pretty","elements","structure", "volume", 'elements'
                                          , "nelements","symmetry"
                                        ,"total_magnetization","total_magnetization_normalized_vol"
                                        ,"total_magnetization_normalized_formula_units"
                                   ,"nsites","material_id"]) 
for entry in tqdm(docFe):
    Formulas.append(entry.formula_pretty)
    material_id.append(entry.material_id)
    total_mag_norm_fu.append(entry.total_magnetization_normalized_formula_units)
    total_mag_norm_vol.append(entry.total_magnetization_normalized_vol)
    total_mag.append(entry.total_magnetization)
    crystal_system.append(entry.symmetry.crystal_system)
    volume.append(entry.volume)
    nelements.append(entry.nelements)
    number.append(entry.symmetry.number)
    symbol.append(entry.symmetry.symbol)
    point_group.append(entry.symmetry.point_group)
    nsites.append(entry.nsites)
    stru.append(entry.structure)
    JarvisFeatures.append(jarvis.featurize(entry.structure))
    
df=pd.DataFrame(JarvisFeatures)
df.columns = jarvis.feature_labels()
df['material_id'] = material_id
df['formula'] = Formulas
df['total_magnetization_normalized_formula_units'] = total_mag_norm_fu
df['total_magnetization_normalized_vol'] = total_mag_norm_vol
df['total_magnetization'] = total_mag
df['volume'] = volume
df['nelements'] = nelements
df['nsites'] = nsites
df['crystal_system'] = crystal_system
df['number'] = number
df['symbol'] = symbol
df['point_group'] = point_group
df['structure'] = stru

df.to_csv(r'C:\Users\joeya\Documents\Fokwa Group\ML_tutorial\intermetallics\intermetallics_dataset_JarvisCrMnFe.csv', index=False)

docCo = mpr.materials.summary.search(elements=["Co"], exclude_elements=["O",'Eu','Tc','Ac',"Pu","Pm","U","Np","Pa","Pu","Th",'H','S','F','Cl','Br','I','At','Xe'] 
                                 ,fields=["formula_pretty","elements","structure", "volume", 'elements'
                                          , "nelements","symmetry"
                                        ,"total_magnetization","total_magnetization_normalized_vol"
                                        ,"total_magnetization_normalized_formula_units"
                                   ,"nsites","material_id"]) 
for entry in tqdm(docCo):
    Formulas.append(entry.formula_pretty)
    material_id.append(entry.material_id)
    total_mag_norm_fu.append(entry.total_magnetization_normalized_formula_units)
    total_mag_norm_vol.append(entry.total_magnetization_normalized_vol)
    total_mag.append(entry.total_magnetization)
    crystal_system.append(entry.symmetry.crystal_system)
    volume.append(entry.volume)
    nelements.append(entry.nelements)
    number.append(entry.symmetry.number)
    symbol.append(entry.symmetry.symbol)
    point_group.append(entry.symmetry.point_group)
    nsites.append(entry.nsites)
    stru.append(entry.structure)
    JarvisFeatures.append(jarvis.featurize(entry.structure))
    
df=pd.DataFrame(JarvisFeatures)
df.columns = jarvis.feature_labels()
df['material_id'] = material_id
df['formula'] = Formulas
df['total_magnetization_normalized_formula_units'] = total_mag_norm_fu
df['total_magnetization_normalized_vol'] = total_mag_norm_vol
df['total_magnetization'] = total_mag
df['volume'] = volume
df['nelements'] = nelements
df['nsites'] = nsites
df['crystal_system'] = crystal_system
df['number'] = number
df['symbol'] = symbol
df['point_group'] = point_group
df['structure'] = stru

df.to_csv(r'C:\Users\joeya\Documents\Fokwa Group\ML_tutorial\intermetallics\intermetallics_dataset_JarvisCrMnFeCo.csv', index=False)

docNi = mpr.materials.summary.search(elements=["Ni"], exclude_elements=["O",'Eu','Tc','Ac',"Pu","Pm","U","Np","Pa","Pu","Th",'H','S','F','Cl','Br','I','At','Xe'] 
                                 ,fields=["formula_pretty","elements","structure", "volume", 'elements'
                                          , "nelements","symmetry"
                                        ,"total_magnetization","total_magnetization_normalized_vol"
                                        ,"total_magnetization_normalized_formula_units"
                                   ,"nsites","material_id"]) 
for entry in tqdm(docNi):
    Formulas.append(entry.formula_pretty)
    material_id.append(entry.material_id)
    total_mag_norm_fu.append(entry.total_magnetization_normalized_formula_units)
    total_mag_norm_vol.append(entry.total_magnetization_normalized_vol)
    total_mag.append(entry.total_magnetization)
    crystal_system.append(entry.symmetry.crystal_system)
    volume.append(entry.volume)
    nelements.append(entry.nelements)
    number.append(entry.symmetry.number)
    symbol.append(entry.symmetry.symbol)
    point_group.append(entry.symmetry.point_group)
    nsites.append(entry.nsites)
    stru.append(entry.structure)
    JarvisFeatures.append(jarvis.featurize(entry.structure))
    
df=pd.DataFrame(JarvisFeatures)
df.columns = jarvis.feature_labels()
df['material_id'] = material_id
df['formula'] = Formulas
df['total_magnetization_normalized_formula_units'] = total_mag_norm_fu
df['total_magnetization_normalized_vol'] = total_mag_norm_vol
df['total_magnetization'] = total_mag
df['volume'] = volume
df['nelements'] = nelements
df['nsites'] = nsites
df['crystal_system'] = crystal_system
df['number'] = number
df['symbol'] = symbol
df['point_group'] = point_group
df['structure'] = stru

print(df.shape)
df.to_csv(r'C:\Users\joeya\Documents\Fokwa Group\ML_tutorial\intermetallics\intermetallics_dataset_Jarvis_Full2.csv', index=False)

C:\Users\joeya\anaconda3\Lib\site-packages\mp_api\client\mprester.py:227: UserWarning: mpcontribs-client not installed. Install the package to query MPContribs data, or construct pourbaix diagrams: 'pip install mpcontribs-client'
  warnings.warn(


['builder_meta', 'nsites', 'elements', 'nelements', 'composition', 'composition_reduced', 'formula_pretty', 'formula_anonymous', 'chemsys', 'volume', 'density', 'density_atomic', 'symmetry', 'property_name', 'material_id', 'deprecated', 'deprecation_reasons', 'last_updated', 'origins', 'warnings', 'structure', 'task_ids', 'uncorrected_energy_per_atom', 'energy_per_atom', 'formation_energy_per_atom', 'energy_above_hull', 'is_stable', 'equilibrium_reaction_energy_per_atom', 'decomposes_to', 'xas', 'grain_boundaries', 'band_gap', 'cbm', 'vbm', 'efermi', 'is_gap_direct', 'is_metal', 'es_source_calc_id', 'bandstructure', 'dos', 'dos_energy_up', 'dos_energy_down', 'is_magnetic', 'ordering', 'total_magnetization', 'total_magnetization_normalized_vol', 'total_magnetization_normalized_formula_units', 'num_magnetic_sites', 'num_unique_magnetic_sites', 'types_of_magnetic_species', 'k_voigt', 'k_reuss', 'k_vrh', 'g_voigt', 'g_reuss', 'g_vrh', 'universal_anisotropy', 'homogeneous_poisson', 'e_total

Retrieving SummaryDoc documents:   0%|          | 0/1434 [00:00<?, ?it/s]

  0%|          | 0/1434 [00:00<?, ?it/s]

Retrieving SummaryDoc documents:   0%|          | 0/2313 [00:00<?, ?it/s]

...

Retrieving SummaryDoc documents:   0%|          | 0/2893 [00:00<?, ?it/s]

  0%|          | 0/2893 [00:00<?, ?it/s]

Retrieving SummaryDoc documents:   0%|          | 0/3592 [00:00<?, ?it/s]

  0%|          | 0/3592 [00:00<?, ?it/s]

(13031, 1570)


In [ ]:
with MPRester(API_KEY) as mpr:
    docCr = mpr.materials.summary.search(elements=["Cr"], exclude_elements=["O",'Eu','Tc','Ac',"Pu","Pm","U","Np","Pa","Pu","Th",'H','S','F','Cl','Br','I','At','Xe'] 
                                 ,fields=["structure",'formula_pretty']) 
for entry in docCr:
    Formulas.append(entry.formula_pretty)
    JarvisFeatures.append(jarvis.featurize(entry.structure))
    